In [1]:
from summoner_data import SummonerData
from map_helper import MapHelper
from recommender import Recommender

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from collab_filter import ChampionsDataset, DotProduct, train_and_evaluate_model
from torch import optim

In [2]:
sd = SummonerData()
a = sd.aggregate_json(True)

Re-aggregating json...
Successfully pickled combined puuid_dict: _FBspdXVQL7lUA3pC25rj6JCEmMa3euPPkzJhdx01EAFTjJ3Z2o4Hxzg5YRmwTGyJMsIOPB17mAbaw


In [5]:
r

,Katarina,Akali,Senna,Vladimir,Vayne,Riven,Tristana,Jayce,XinZhao,Zeri,...,Quinn,Malzahar,Singed,Nasus,Amumu,Shyvana,Heimerdinger,Renata,Urgot,Belveth
-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ,0.0,0.000000,1.695876,1.210699,2.712990,6.171372,0.000000,0.000000,0.604157,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
0EMIdPXGT4kuqhWoMhAyVcfuaHlRk87Ofm-KuAtYVKkcxxls6RNZ-Go1XJcVX6n9jsI30dmzNQ_PVw,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.628150,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
0EpXofpvMfZ4N-4maPmTOEfGvewDW4zJQAQgbxPZkWU4LOQY7zcUE8WuaOtwqAh2W6VlDqOz9JujOQ,0.0,0.000000,0.000000,0.000000,0.637745,0.000000,3.380902,1.012780,0.000000,3.905886,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
0pMjPtiynYXJ3pmdUtfhKfbdTp8dUIUVE5PK4bXmCl2frl8g4io9rqO2nTAT2YAQv-W5Sk267G2uLA,0.0,0.000000,1.501905,0.000000,0.000000,0.000000,0.000000,0.000000,4.248315,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
0ujxq14s8XtEC3gkD9ixea1NWVmUHvPq8CnZpEMAU3WrlNNpqWA87Izne0l2ucpP772dozhQqh9NCQ,0.0,0.000000,1.069979,0.000000,0.000000,0.000000,0.000000,1.501905,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEu2VmDjTs7vEGGnijkhGOc_pVjl44RLyHgelCGiOG1FZ8hWgFJIWq8Mcuk7Bb8aK09bxAY4AVoyIw,0.0,0.000000,1.302267,0.000000,0.000000,0.000000,2.168593,0.000000,0.000000,1.657037,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ZHV8Cg7rbHx-auwZGbb2GltYYkY9jwKNz4NUVkE228QVWFIhcvfbH-xaATOg2c0eaxedXNKnGShMIQ,0.0,3.197317,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.356761,0.681051,0.0,0.0,0.0,0.0,0.0
ZoZW-S88lemcElWSZpq0gSQG_DHDeiL30kg_5XEXalHfVqi2rhRDojhknodpOQwS9rU5DS_2NYQhXA,0.0,0.000000,0.000000,0.810280,0.000000,0.000000,1.501905,1.566987,0.262751,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
_cE7uj524OI1EwETD7aGRHqjAs0S3Bcn3rI4yWk7ofmejPfGBSuWGelrTfuvTB1Zjgzq8hmx8JgwaA,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [6]:
r

,55,84,235,8,67,92,18,126,5,221,...,75,63,32,102,74,888,44,6,200,350
-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ,0.000000,0.000000,1.199864,0.757030,1.993322,4.336679,0.000000,0.000000,0.565292,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
0EMIdPXGT4kuqhWoMhAyVcfuaHlRk87Ofm-KuAtYVKkcxxls6RNZ-Go1XJcVX6n9jsI30dmzNQ_PVw,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.510842,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
0EpXofpvMfZ4N-4maPmTOEfGvewDW4zJQAQgbxPZkWU4LOQY7zcUE8WuaOtwqAh2W6VlDqOz9JujOQ,0.000000,0.000000,0.000000,0.000000,0.793488,0.000000,2.996600,0.818809,0.000000,4.516911,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
0pMjPtiynYXJ3pmdUtfhKfbdTp8dUIUVE5PK4bXmCl2frl8g4io9rqO2nTAT2YAQv-W5Sk267G2uLA,0.000000,0.000000,0.289198,0.000000,0.000000,0.000000,0.000000,0.000000,1.997887,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
0ujxq14s8XtEC3gkD9ixea1NWVmUHvPq8CnZpEMAU3WrlNNpqWA87Izne0l2ucpP772dozhQqh9NCQ,0.000000,0.000000,0.794687,0.000000,0.000000,0.000000,0.000000,0.780780,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
1qpy4uFGix-edD-PFZyoCm36CUsjT3MY4aJNTZWR_bl17KOjHjaHSTd16sn-XguntGTi1PauSwXoMw,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2VTuAwGYOtR4I-Fxz650zkcbWzLYotgblnzCy9608WmSxNsWeVT6EhRDypWZKmxrLQGWiEyc5q_NEA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.516866,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3E_pbXhHAHTOA8tgijXm2lSCsL4zSoFnl10PRGM5QDMkn_uaYK5K_TEkeJ9oz-6DpHcarR3GBw1dfA,0.000000,0.000000,0.000000,2.382197,0.000000,0.000000,0.000000,2.132327,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3v44PbOGflb21JDbMcak5S9GfbcLgGWbtzkY7rE7Akanq94OqTwBTUOZhC19MLvMlYcWQHMGmaV8QQ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
51FrqP2j67SB3e56cq-6D01616QNyUjtFLQCpWpVmqA08wlq0DpsPRu5ecp1GFO26SimmyNI144O0g,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.542185,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
import os
pr = os.getenv("PROJECT_ROOT")

json_folder_path = os.path.join(pr, "src/summoner_data/")

# {player_id: {champion_id: count}}
# player_champions = defaultdict(lambda: defaultdict(int))

combined_puuid_dict = {}
for json_file in os.listdir(json_folder_path):
    puuid = json_file.split(".json")[0]
    s = sd.load_pkl_from_json(puuid)
    # print(puuid)

In [7]:
combined_puuid_dict = {}
for json_file in os.listdir(json_folder_path):
    puuid = json_file.split(".json")[0]
    combined_puuid_dict.update(
        sd.load_pkl_from_json(puuid)
    )

In [11]:
combined_puuid_dict.keys()

dict_keys(['-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ', '0EMIdPXGT4kuqhWoMhAyVcfuaHlRk87Ofm-KuAtYVKkcxxls6RNZ-Go1XJcVX6n9jsI30dmzNQ_PVw', '0EpXofpvMfZ4N-4maPmTOEfGvewDW4zJQAQgbxPZkWU4LOQY7zcUE8WuaOtwqAh2W6VlDqOz9JujOQ', '0pMjPtiynYXJ3pmdUtfhKfbdTp8dUIUVE5PK4bXmCl2frl8g4io9rqO2nTAT2YAQv-W5Sk267G2uLA', '0ujxq14s8XtEC3gkD9ixea1NWVmUHvPq8CnZpEMAU3WrlNNpqWA87Izne0l2ucpP772dozhQqh9NCQ', '1qpy4uFGix-edD-PFZyoCm36CUsjT3MY4aJNTZWR_bl17KOjHjaHSTd16sn-XguntGTi1PauSwXoMw', '2VTuAwGYOtR4I-Fxz650zkcbWzLYotgblnzCy9608WmSxNsWeVT6EhRDypWZKmxrLQGWiEyc5q_NEA', '3E_pbXhHAHTOA8tgijXm2lSCsL4zSoFnl10PRGM5QDMkn_uaYK5K_TEkeJ9oz-6DpHcarR3GBw1dfA', '3v44PbOGflb21JDbMcak5S9GfbcLgGWbtzkY7rE7Akanq94OqTwBTUOZhC19MLvMlYcWQHMGmaV8QQ', '4rDB2_T6QsTCrGxxu2hZ4XhT53DNuyGCEc0Kbh9cq49_Peq3yeMJvsTxo6eP1RgMrdeP5lzEBt5VRQ', '51FrqP2j67SB3e56cq-6D01616QNyUjtFLQCpWpVmqA08wlq0DpsPRu5ecp1GFO26SimmyNI144O0g', '59wIBFUGdIyAV0mDgvOff8ue2RXlObrchIy6FMrDXNiP3O53Hbnh7iGb0StqLLkBuABeeHsLQ5C8bA', '5fPn

In [3]:
s = sd.aggregate_json(True)
s

Re-aggregating json...


{'-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ': defaultdict(int,
             {'Katarina': 1,
              'Katarina_kills': 4,
              'Katarina_assists': 4,
              'Katarina_deaths': 6,
              'Akali': 2,
              'Akali_kills': 10,
              'Akali_assists': 13,
              'Akali_deaths': 10,
              'Senna': 2,
              'Senna_kills': 18,
              'Senna_assists': 18,
              'Senna_deaths': 12,
              'Senna_wins': 2,
              'Vladimir': 1,
              'Vladimir_kills': 10,
              'Vladimir_assists': 9,
              'Vladimir_deaths': 5,
              'Vladimir_wins': 1,
              'Vayne': 18,
              'Vayne_kills': 178,
              'Vayne_assists': 113,
              'Vayne_deaths': 74,
              'Vayne_wins': 10,
              'Riven': 39,
              'Riven_kills': 295,
              'Riven_assists': 223,
              'Riven_deaths': 141,
         

In [8]:
for a, b in champ_data.items():
    if '_' not in a:
        print(a)

Draven
Vayne
Aphelios
Leona
Graves
Varus
Seraphine
Olaf
Jhin
Darius
Karthus
Caitlyn
Kaisa
LeeSin
Twitch
Jayce
Ezreal
Lucian
TwistedFate


In [4]:
sd = SummonerData()
mh = MapHelper()
r = Recommender()

In [ ]:
df, le_user, le_champion = sd.load_clean_df_encoders(True)

# train test split
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader= DataLoader(test_dataset, batch_size=20, shuffle=True)

num_summoners = len(df['puuid'].unique())
num_champions = len(df['champ_id'].unique())
num_factors = 5

In [9]:
model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader)

Epoch 1, Loss: 4.906643151940398
Test MSE: 4.873281983939184
Epoch 2, Loss: 4.735216017979295
Test MSE: 4.715137396992274
Epoch 3, Loss: 4.563998546469864
Test MSE: 4.5647966227125645
Epoch 4, Loss: 4.408914638247656
Test MSE: 4.421540078197703
Epoch 5, Loss: 4.263659830413648
Test MSE: 4.284854095181185
Epoch 6, Loss: 4.125717077682268
Test MSE: 4.154301305720668
Epoch 7, Loss: 3.9918327395299182
Test MSE: 4.029618926154049
Epoch 8, Loss: 3.8692212674155164
Test MSE: 3.9105257482809996
Epoch 9, Loss: 3.738593636905376
Test MSE: 3.7966753531934687
Epoch 10, Loss: 3.624659344924623
Test MSE: 3.6876631279207994
Epoch 11, Loss: 3.5090813630848974
Test MSE: 3.5833969698168375
Epoch 12, Loss: 3.4175640961424034
Test MSE: 3.482982299272616
Epoch 13, Loss: 3.3110438511146243
Test MSE: 3.386856639463532
Epoch 14, Loss: 3.211867269295365
Test MSE: 3.294872452000721
Epoch 15, Loss: 3.1236244831512225
Test MSE: 3.20654004649991
Epoch 16, Loss: 3.023685278732385
Test MSE: 3.1216908251901336
Epoch 

In [10]:
all_champions = torch.tensor([i for i in range(num_champions)])
user_id = torch.tensor([1] * len(all_champions)) 
predicted_ratings = model(user_id, all_champions)
predicted_ratings

tensor([-0.5947,  1.6177, -0.8919,  1.2380,  1.1635, -0.2104,  0.4658,  1.0459,
         1.3852,  0.2217, -1.5988, -1.0558,  2.5141, -0.6322, -0.4344, -1.1053,
        -1.7794,  1.5128, -0.7779, -0.7559,  0.2886,  0.7452, -2.2807,  0.8790,
         0.7902,  0.2404, -0.2707,  0.4557, -0.8077,  0.0375, -1.5497, -0.0630,
         0.7642, -0.2915, -0.0761,  0.0705, -0.2367,  0.2173,  0.7507, -0.5542,
         0.0614,  0.2518, -1.3512,  0.0602,  1.1121, -0.7615,  0.0329, -0.8246,
         0.9438, -0.5896,  1.5078, -0.5570,  0.0637,  0.4021, -0.5378, -0.4403,
         1.5223,  0.4493,  1.7083, -1.3657,  0.0434,  0.7818,  0.5214,  0.0951,
        -0.6546,  1.4233,  1.1007,  0.1599, -0.7224, -1.5735, -0.1484, -0.9160,
        -1.2607, -0.5429, -0.8551,  0.7927,  1.0763,  0.7488, -0.4731, -0.4628,
         0.0579,  0.3904, -1.0221, -0.3261,  0.6984, -0.3940,  0.0033,  0.5445,
        -0.1851,  0.4334,  0.3017,  0.1579,  0.6551, -0.7888,  0.0537,  0.6225,
         0.5042,  0.8954, -0.3772,  0.40

In [11]:
top_ratings, top_indices = torch.topk(predicted_ratings, k=3)

# Convert tensor indices to integers if needed
top_indices_list = top_indices.tolist()

In [12]:
for idx in top_indices_list:
    print(f"WE PRESCRIBE THAT YOU PLAY: {champ_order[idx]}")

NameError: name 'champ_order' is not defined

In [ ]:
# WHAT IS YOUR SUMMONER ID? 
# YOU ARE NOT ON FILE. GENERATING...
# YOU ARE ON FILE. MODELLING...
# YOUR CURRENT MOST PLAYED IS ... WE RECOMMEND...

In [4]:
le_user.inverse_transform([1])

array(['0EMIdPXGT4kuqhWoMhAyVcfuaHlRk87Ofm-KuAtYVKkcxxls6RNZ-Go1XJcVX6n9jsI30dmzNQ_PVw'],
      dtype=object)

In [26]:
print("Print your in game name as such: Fred#NA1...")

Print your in game name as such: Fred#NA1...


In [6]:
import os
RIOT_API_KEY = os.environ.get("RIOT_API_KEY")
RIOT_API_KEY

'RGAPI-67a9e6ce-05b9-41a9-a84f-6f4ac2b7d2ff'

In [1]:
# Puts it all together
import os

import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from collab_filter import ChampionsDataset, DotProduct, train_and_evaluate_model
from map_helper import MapHelper
from summoner_data import SummonerData

PROJECT_ROOT = os.getenv("PROJECT_ROOT")

map_helper = MapHelper()
summoner_data = SummonerData()
rewrite = False

# Prompt in game name
puuid = map_helper.get_puuid_mapping()

In [ ]:
#Check if data exists as json
puuid_path = os.path.join(PROJECT_ROOT, f"src/summoner_data/{puuid}.json")
if not os.path.exists(puuid_path):
    rewrite = True
    summoner_data.dump_matches_for_puuid(puuid)

# Get the rating data
_ = summoner_data.aggregate_json(True)
df, le_user, le_champion = summoner_data.load_clean_df_encoders(True)

# Train a model
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=True)

num_summoners = len(df["puuid"].unique())
num_champions = len(df["champ_id"].unique())
num_factors = 5

model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader)
# Get relevant row
user_idx = le_user.transform([puuid])

Re-aggregating json...
Loading new ratings...
User Embeddings Before Training:
tensor([[ 2.3690,  0.2873,  0.8923, -0.2729, -0.6466],
        [-0.1189, -0.8681, -1.4192,  1.1580,  0.5485],
        [ 0.0672, -1.2097,  0.0176,  0.6157, -0.6444],
        [-0.2056,  1.5893, -0.2634, -0.2865, -1.3269],
        [-0.1026,  0.1347,  0.6852, -1.3550, -0.7912]],
       grad_fn=<SliceBackward0>)
Item Embeddings Before Training:
tensor([[ 0.0249,  0.3249,  0.7492,  0.1573, -1.8750],
        [ 0.4295, -0.5778,  0.6961,  1.0842,  1.3802],
        [ 0.5694,  0.8366,  1.4554, -0.2050,  1.1136],
        [ 0.3418, -0.8622, -0.1369, -0.1809,  1.6811],
        [-1.3125, -0.0887, -1.0633,  0.1687, -0.6163]],
       grad_fn=<SliceBackward0>)
Epoch 1, Loss: 4.771667809925269
Test MSE (Rounded Predictions): 4.563136100769043
Epoch 2, Loss: 4.5973392962223265
Test MSE (Rounded Predictions): 4.414419651031494
Epoch 3, Loss: 4.441386195261087
Test MSE (Rounded Predictions): 4.31016731262207
Epoch 4, Loss: 4.2897

In [15]:
all_champions

tensor([0.6734, 0.8631, 0.9283, 0.9599, 0.6041, 0.9548, 0.5921, 0.1938, 0.2579,
        0.9226, 0.6161, 0.1246, 0.8867, 0.4801, 0.0662, 0.0833, 0.6780, 0.1933,
        0.1752, 0.6738, 0.3451, 0.8650, 0.2480, 0.7855, 0.0153, 0.6244, 0.1878,
        0.5202, 0.2650, 0.2060, 0.3161, 0.4936, 0.1617, 0.4068, 0.3938, 0.9145,
        0.8185, 0.8273, 0.4854, 0.8114, 0.6261, 0.0647, 0.1143, 0.6823, 0.4623,
        0.8374, 0.6811, 0.1737, 0.1535, 0.2488, 0.0855, 0.6467, 0.4845, 0.8451,
        0.0297, 0.7419, 0.7130, 0.9670, 0.6072, 0.5096, 0.1103, 0.1724, 0.7426,
        0.4234, 0.4896, 0.9261, 0.3896, 0.6541, 0.3440, 0.3871, 0.5896, 0.7940,
        0.8262, 0.6594, 0.9704, 0.2902, 0.6415, 0.9639, 0.1097, 0.1480, 0.2111,
        0.8691, 0.2314, 0.8650, 0.3334, 0.4526, 0.7835, 0.4966, 0.9833, 0.6980,
        0.8537, 0.6218, 0.8743, 0.7042, 0.4063, 0.5714, 0.8051, 0.7978, 0.3211,
        0.4429, 0.4833, 0.2745, 0.2715, 0.1345, 0.2682, 0.3595, 0.1886, 0.1775,
        0.3104, 0.5436, 0.5687, 0.0617, 

In [25]:
import numpy as np
# Get pred
all_champions = torch.tensor([i for i in range(num_champions)])
# all_champions = torch.tensor(df['champ_id'].unique(), dtype=torch.long)
# user_id = torch.tensor(list(user_idx) * len(all_champions))
user_id = torch.tensor([1] * len(all_champions))
predicted_ratings = model(user_id, all_champions)
top_ratings, top_indices = torch.topk(predicted_ratings, k=3)
top_indices_list = top_indices.tolist()

# Load champ order
champ_map = map_helper.get_champ_id_to_name()
champ_order = [
    champ_map.get(str(x)) for x in le_champion.inverse_transform(range(num_champions))
]
for idx in top_indices_list:
    print(f"WE PRESCRIBE THAT YOU PLAY: {champ_order[idx]}")

WE PRESCRIBE THAT YOU PLAY: Viktor
WE PRESCRIBE THAT YOU PLAY: Yone
WE PRESCRIBE THAT YOU PLAY: Malphite


In [7]:
model.summoner_factors.weight[:5]

tensor([[-0.8350,  0.9490,  0.3413,  1.0128,  1.0120],
        [ 0.8737, -0.4630, -1.4928, -0.2391,  0.9873],
        [ 1.1687,  0.3299,  0.0453, -0.5464, -1.0530],
        [-0.5251, -0.1215,  1.0812, -0.3236,  0.3960],
        [-1.0846,  0.1898,  0.2224,  0.2006, -0.1719]],
       grad_fn=<SliceBackward0>)

In [6]:
user_id = torch.tensor([3] * len(all_champions))
predicted_ratings = model(user_id, all_champions)
predicted_ratings

tensor([0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877, 0.9877,
        0.9877, 0.9877, 0.9877, 0.9877, 

In [46]:
import requests

RIOT_API_KEY = os.environ.get("RIOT_API_KEY")
game_name = "BSIZZLEMONEY"
tag_line = "0000"

url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
headers = {"X-Riot-Token": RIOT_API_KEY}

response = requests.get(url, headers=headers)
print(response.status_code)
print(response.json())


400
{'status': {'message': 'Unknown apikey', 'status_code': 400}}


In [12]:
le_champion.inverse_transform(range(num_champions))

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 53, 54, 55, 56, 57,
       58, 59, 60, 61, 62, 63, 64, 67, 68, 69, 72, 74, 75, 76, 77, 78, 79,
       80, 81, 82, 83, 84, 85, 86, 89, 90, 91, 92, 96, 98, 99, 101, 102,
       103, 104, 105, 106, 107, 110, 111, 112, 113, 114, 115, 117, 119,
       120, 121, 122, 126, 127, 131, 133, 134, 136, 141, 142, 143, 145,
       147, 150, 154, 157, 161, 163, 164, 166, 200, 201, 202, 203, 221,
       222, 223, 233, 234, 235, 236, 238, 240, 245, 246, 254, 266, 267,
       268, 350, 360, 412, 420, 421, 427, 429, 432, 497, 498, 516, 517,
       518, 523, 526, 555, 711, 777, 799, 800, 875, 876, 887, 888, 893,
       895, 897, 901, 902, 910, 950], dtype=object)

In [ ]:
from map_helper import get_champ_id_to_name
map = get_champ_id_to_name()
champ_order = [map.get(str(x)) for x in le_champion.inverse_transform(range(num_champions))]

In [ ]:
"""
for epoch in range(epochs):
    # set in training mode
    model.train()
    # some metric of keeping track of losses
    loss = 0
    criterion = nn.MSELoss()
    # these are tensors, in this case the locations of whichever data batch is being handed over
    for user_id, champ_id, ratings in train_loader:
        # Just empties the gradients before
        optimizer.zero_grad()
        # this gets passed into the forward function
        preds = model(user_id, champ_id)
        # compares yhat to y
        loss = criterion(preds, ratings)

        # Computes the dloss/dx for every parameter x... what is x?... so something like x.grad += dloss/dx 
        loss.backward()
        # updates the x value using x.grad... so x += -lr * x.grad
        optimizer.step()
        loss += loss.item()

        
        
        
        
        
Synapse that transforms the X into something... and each node then puts an activation function (think sigmoid)
Think of this as the W in the matmul, except this time we're literally just amking it a... dot product?
gradient descent is saying do we want W to go... up or down? gives us a direction to minimize cost
Back prop is essentially saying that we are starting from the end and the "error" term of gradient descent is passed on to the node which contributes the most
every example gets a vote on which way is downhill        
"""

One layer
    Synapse takes the dot product 

Main confusing parts are loss.backward() and optimizer.step() (what are the params excatly?) as well as what's going on in the forward.... like what are user_id and champ_id representing? 

In [ ]:
import json
import os
import pickle as pkl
import time
from collections import defaultdict
from typing import Any, Dict, List, Optional

import pandas as pd
import requests
from sklearn.preprocessing import LabelEncoder

from map_helper import get_champ_id_to_name

API_KEY = os.getenv("RIOT_API_KEY")
PROJECT_ROOT = "/Users/brtnl/OneDrive/Desktop/code/lol_main_recommender"
HEADERS = {"X-Riot-Token": API_KEY}
REGION = "na1"
MATCH_REGION = "americas"

pkl_path = os.path.join(PROJECT_ROOT, "src/data.pkl")

json_folder_path = os.path.join(PROJECT_ROOT, "src/summoner_data/")

player_champions = defaultdict(
    lambda: defaultdict(int)
)  # {player_id: {champion_id: count}}
for json_file in os.listdir(json_folder_path):
    puuid = json_file.split(".json")[0]
    with open(os.path.join(json_folder_path, json_file), "r") as f:
        match_data = json.load(f)
    for match in match_data:
        participants = match.get("info", {}).get("participants", [])
        for participant in participants:
            if participant.get("puuid") == puuid:
                # Store information to get rating
                champion_id = participant.get("championId")
                win = participant.get("win")
                kills = participant.get("kills")
                deaths = participant.get("deaths")
                assists = participant.get("assists")

                # rating = (win_percentage_weight * win_percentage) + (kda_weight * kda) + (matches_played_weight * matches_played)
                player_champions[puuid][champion_id] += 1
                player_champions[puuid][f"{champion_id}_kills"] += kills
                player_champions[puuid][f"{champion_id}_assists"] += assists
                player_champions[puuid][f"{champion_id}_deaths"] += deaths
                if win == 1:
                    player_champions[puuid][f"{champion_id}_wins"] += 1

In [26]:
player_champion_stats = defaultdict(lambda: defaultdict(int))
# calculate kda
for puuid, champ_data in player_champions.items():
    for champion_id, count in champ_data.items():
        # If the key is equal to the id, can construct the stats
        if isinstance(champion_id, int):
            kills = player_champions[puuid][f"{champion_id}_kills"]
            assist = player_champions[puuid][f"{champion_id}_assists"]
            deaths = player_champions[puuid][f"{champion_id}_deaths"]
            wins = player_champions[puuid].get(f"{champion_id}_wins", 0)
            win_pct = (wins / count) * 100 if count > 0 else 0
            kda = (kills + assists) / (deaths if deaths != 0 else 1)

            player_champion_stats[puuid][champion_id] = kda + win_pct + count

In [27]:
player_champion_stats

defaultdict(<function __main__.<lambda>()>,
            {'-zSba7M6_SDDJD1sJ3n2jYDih8mLAawHOXA-hNB554wLALfsNcCPUsrEyaB-7qg8MY3rLQhYQB0KmQ': defaultdict(int,
                         {55: 3.0,
                          84: 3.0769230769230766,
                          235: 103.22222222222223,
                          8: 102.55555555555556,
                          67: 75.1661750245821,
                          92: 109.57157640565711,
                          18: 6.0,
                          126: 31.272727272727273,
                          5: 54.45454545454545,
                          221: 12.0,
                          98: 102.2,
                          91: 101.71428571428571,
                          21: 104.55555555555556,
                          523: 111.5,
                          64: 8.5,
                          238: 102.5,
                          37: 101.5,
                          266: 109.5,
                          51: 7.0,
                          76: 3.